# NOTEBOOK COPIA EJEMPLO NIXTLA hierarchicalforecast

----
### ejemplo conciliacion GEOGRÁFICA Y TEMPORAL de los datos
- Ejecución de notebook ejemplo base de nixtla - replicar código ejemplo y análisis extra para entender código y modelos ya hechos

- EJEMPLO LA DATA FULL DE TOURISM

- FUENTES:
    - geographical aggregation (POINT CONCILIATION): https://nixtlaverse.nixtla.io/hierarchicalforecast/examples/australiandomestictourism.html

    - temporal aggregation (TEMPORAL CONCILIATION): https://nixtlaverse.nixtla.io/hierarchicalforecast/examples/australiandomestictourismtemporal.html
 
    - geographical and temporal aggregation: https://nixtlaverse.nixtla.io/hierarchicalforecast/examples/australiandomestictourismcrosstemporal.html
 
    - paper base investigación (los códigos de nixtla replican lo obtenido por el paper): https://robjhyndman.com/seminars/fr_overview.html

# GEOGRÁFICA Y TEMPORAL RECONCILIATION (Tourism)
FUENTE BASE: https://nixtlaverse.nixtla.io/hierarchicalforecast/examples/australiandomestictourismcrosstemporal.html

We will first load the Tourism data and produce base forecasts using an AutoETS model from StatsForecast. Then, we reconciliate the forecasts with several reconciliation algorithms from HierarchicalForecast according to the cross-sectional geographical hierarchies. Finally, we reconciliate the forecasts in the temporal dimension according to a temporal hierarchy.

#### ESTE NOTEBOOK ES LA CONSOLIDACIÓN DE LOS 2 NOTEBOOKS ANTERIORES, SE REALIZA PRIMERO UNA CONSOLIDACIÓN GEOGRÁFICA Y LUEGO CONSOLIDACIÓN TEMPORAL

### 0. Install nixtla package

In [1]:
# pip install hierarchicalforecast
# pip install datasetsforecast
# pip install statsforecast

In [2]:
import hierarchicalforecast
from datasetsforecast.hierarchical import HierarchicalData
import pandas as pd

## run code - example nixtla

In [3]:
import numpy as np
import pandas as pd

In [4]:
# librería para generar los datos desde los niveles de mayor desagracion, hacer las agregaciones ej data trimestral a data anual
from hierarchicalforecast.utils import aggregate
from hierarchicalforecast.utils import aggregate_temporal

In [5]:
from statsforecast.models import AutoETS
from statsforecast.core import StatsForecast

In [6]:
from hierarchicalforecast.methods import BottomUp, MinTrace
from hierarchicalforecast.core import HierarchicalReconciliation

### 1. Read data - download data example hyndman

#### 1.1 read raw data

In [7]:
data_raw = pd.read_csv('https://raw.githubusercontent.com/Nixtla/transfer-learning-time-series/main/datasets/tourism.csv')
data_raw = data_raw.rename({'Trips': 'y', 'Quarter': 'ds'}, axis=1)
data_raw.insert(0, 'Country', 'Australia')
data_raw = data_raw[['Country', 'Region', 'State', 'Purpose', 'ds', 'y']]
data_raw['ds'] = data_raw['ds'].str.replace(r'(\d+) (Q\d)', r'\1-\2', regex=True)
data_raw['ds'] = pd.PeriodIndex(data_raw["ds"], freq='Q').to_timestamp()

In [8]:
# print data - se observa que a diferencia del ejemplo small que se carga la matriz S para realizar la reconciliación de forecast
# aquí estan todas las series y hay que armar la matriz de cómo se relacionan los forecasts
data_raw.head()

,Country,Region,State,Purpose,ds,y
0,Australia,Adelaide,South Australia,Business,1998-01-01,135.077690
1,Australia,Adelaide,South Australia,Business,1998-04-01,109.987316
2,Australia,Adelaide,South Australia,Business,1998-07-01,166.034687
3,Australia,Adelaide,South Australia,Business,1998-10-01,127.160464
4,Australia,Adelaide,South Australia,Business,1999-01-01,137.448533


### 2. Realizar agregación de los datos - geográfica - para hacer fcst todas las agregaciones geográficas (ej por región y total país)
##### Aggregating the dataset according to cross-sectional hierarchy "(CS)"

In [9]:
# matriz que relaciona las diferentes columnas - agregación geográfica
spec = [
    ['Country'],
    ['Country', 'State'], 
    ['Country', 'Purpose'], 
    ['Country', 'State', 'Region'], 
    ['Country', 'State', 'Purpose'], 
    ['Country', 'State', 'Region', 'Purpose']
]

In [10]:
# generar dataframe con las series individuales (data original) y con las series agregadas hasta tener total_site
Y_df_cs, S_df_cs, tags_cs = aggregate(data_raw, spec)

In [11]:
Y_df_cs.head(3)

,unique_id,ds,y
0,Australia,1998-01-01,23182.197269
1,Australia,1998-04-01,20323.380067
2,Australia,1998-07-01,19826.640511


In [12]:
Y_df_cs.tail(3)

,unique_id,ds,y
33997,Australia/Western Australia/Experience Perth/V...,2017-04-01,302.296119
33998,Australia/Western Australia/Experience Perth/V...,2017-07-01,373.442070
33999,Australia/Western Australia/Experience Perth/V...,2017-10-01,455.316702


In [13]:
S_df_cs.head(3)

,unique_id,Australia/ACT/Canberra/Business,Australia/ACT/Canberra/Holiday,Australia/ACT/Canberra/Other,Australia/ACT/Canberra/Visiting,Australia/New South Wales/Blue Mountains/Business,Australia/New South Wales/Blue Mountains/Holiday,Australia/New South Wales/Blue Mountains/Other,Australia/New South Wales/Blue Mountains/Visiting,Australia/New South Wales/Capital Country/Business,...,Australia/Western Australia/Australia's North West/Other,Australia/Western Australia/Australia's North West/Visiting,Australia/Western Australia/Australia's South West/Business,Australia/Western Australia/Australia's South West/Holiday,Australia/Western Australia/Australia's South West/Other,Australia/Western Australia/Australia's South West/Visiting,Australia/Western Australia/Experience Perth/Business,Australia/Western Australia/Experience Perth/Holiday,Australia/Western Australia/Experience Perth/Other,Australia/Western Australia/Experience Perth/Visiting
0,Australia,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,Australia/ACT,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Australia/New South Wales,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
tags_cs.keys()

dict_keys(['Country', 'Country/State', 'Country/Purpose', 'Country/State/Region', 'Country/State/Purpose', 'Country/State/Region/Purpose'])

### 3. Separar train y test

In [15]:
# horizonte fcst a generar
horizon = 8

In [16]:
# split train y test
Y_test_df_cs = Y_df_cs.groupby("unique_id", as_index=False).tail(horizon)
Y_train_df_cs = Y_df_cs.drop(Y_test_df_cs.index)

In [17]:
# print informativo tamaño de la data original, data_train y data_test
print('Y_df_cs: ', Y_df_cs.shape)
print('Y_train_df_cs: ', Y_train_df_cs.shape)
print('Y_test_df_cs: ', Y_test_df_cs.shape)

Y_df_cs:  (34000, 3)
Y_train_df_cs:  (30600, 3)
Y_test_df_cs:  (3400, 3)


### 4. Computing base forecasts (cross sectional datasets)

In [18]:
# modelo base
fcst = StatsForecast(models = [AutoETS(season_length = 4, model = 'ZZA')], 
                     freq = 'QS',
                     n_jobs = -1)

# realizar fcst horizonte futuro
Y_hat_df_cs = fcst.forecast(df = Y_train_df_cs, h = horizon, fitted = True)

# generar data TRAIN con los fcst generados en train (se hace porque una de las consolidaciones necesitan tener el error de train)
Y_fitted_df_cs = fcst.forecast_fitted_values()

### 5. Reconciliar fcst (cross sectional data)

In [19]:
# definir metodos de reconciliación a probar
reconcilers = [
    BottomUp(),
    MinTrace(method='mint_shrink'),
    MinTrace(method='ols')
]

In [20]:
# generar instancia de conciliador (point - cross sectional data)
hrec = HierarchicalReconciliation(reconcilers = reconcilers)
hrec

In [21]:
# realizar conciliacion data cross sectional
Y_rec_df_cs = hrec.reconcile(Y_hat_df = Y_hat_df_cs, # data forecasteada
                             Y_df = Y_fitted_df_cs, # data train con valores y_true, y_pred
                             S = S_df_cs, 
                             tags = tags_cs
                            )

In [22]:
Y_rec_df_cs.head(3)

,unique_id,ds,AutoETS,AutoETS/BottomUp,AutoETS/MinTrace_method-mint_shrink,AutoETS/MinTrace_method-ols
0,Australia,2016-01-01,25990.068004,24381.672902,25427.793552,25894.419896
1,Australia,2016-04-01,24458.490282,22903.194015,23913.800914,24357.231461
2,Australia,2016-07-01,23974.055984,22411.401316,23428.540858,23865.928094


### 6. Validar que se cumple la conservación del volumen (ojo el cálculo realizado es el mismo encontrado en el notebook de SOLO conciliación geográfica)

In [23]:
# obtener las series de "Country"
list_series_country = tags_cs['Country'].tolist()
list_series_country

['Australia']

In [24]:
# obtener las series 'Country/State/Region/Purpose' (EL NIVEL DE MAYOR DESAGREGACIÓN DEL FORECAST)
list_series_all_desagregacion = tags_cs['Country/State/Region/Purpose'].tolist()
len(list_series_all_desagregacion)

304

In [25]:
# definir fecha de fcst para evaluar las diferentes agregaciones geográficas
fecha_example_filter = '2016-01-01'

In [26]:
# filtrar forecast país para una fecha en específico (para comparar vs la suma del forecast cada state por separado y luego sumándolo)
mask_example_country = (Y_rec_df_cs['unique_id'].isin(list_series_country)) & (Y_rec_df_cs['ds'] == fecha_example_filter)
Y_rec_df_cs[mask_example_country]

,unique_id,ds,AutoETS,AutoETS/BottomUp,AutoETS/MinTrace_method-mint_shrink,AutoETS/MinTrace_method-ols
0,Australia,2016-01-01,25990.068004,24381.672902,25427.793552,25894.419896


In [27]:
# FILTRAR FORECAST DEL NIVEL DE MAYOR DESEGREGACION Y SUMAR HASTA LLEGAR A COUNTRY. validar que bottom/up sumó desde el nivel de mayor desagregacion
mask_example_all_desagregation = (Y_rec_df_cs['unique_id'].isin(list_series_all_desagregacion)) & (Y_rec_df_cs['ds'] == fecha_example_filter)
Y_rec_df_example = Y_rec_df_cs[mask_example_all_desagregation]

cols_to_sum = ['AutoETS', 'AutoETS/BottomUp', 'AutoETS/MinTrace_method-mint_shrink', 'AutoETS/MinTrace_method-ols']
Y_rec_df_example[cols_to_sum].sum(axis=0).to_frame(name='sum_result').T

,AutoETS,AutoETS/BottomUp,AutoETS/MinTrace_method-mint_shrink,AutoETS/MinTrace_method-ols
sum_result,24381.672902,24381.672902,25427.793552,25894.419896


In [28]:
# SE VALIDA QUE
# la suma del fsct del mayor grado de desagracion "24381.672902" es igual a la consolidacion bottom/up (evaluando volumen a total pais)
# luego de consolidar el volumen del fcst total_país es igual a la suman de todos los forecast individuales hasta llegar a nivel país
# (lo esperado, que se conserve el volumen, y luego al evaluar métricas debería dar mejores resultados)

### 7. Temporal reconciliation
LUEGO DE HACER CONSOLIDACIÓN GEOGRÁFICA, SE PROCEDE A HACER LA CONSOLIDACIÓN TEMPORAL

**OJO, SE TIENE QUE GENERAR LA DATA CON TODAS LAS AGREGACIONES TEMPORALES Y LUEGO SE HACE UNA CONSOLIDACION "DOBLE" CONSOLIDAR GEOGRÁRICA Y TEMPORALMENTE**

The spec is a dictionary in which the keys are the name of the aggregation and the value is the amount of bottom-level timesteps that should be aggregated in that aggregation. For example, year consists of 12 months, so we define a key, value pair "yearly":12. We can do something similar for other aggregations that we are interested in.

In [29]:
# diccionario de cuantas observaciones se deben de tomar para obtener la agregacion temporal 
# (ej data trimestral, tomando 4 datos se tiene anual)
spec_temporal = {"year": 4, "semiannual": 2, "quarter": 1}
spec_temporal

{'year': 4, 'semiannual': 2, 'quarter': 1}

In [30]:
# print data input - DATA ya agregada/expandida a todos los grupos geográficos
Y_train_df_cs

,unique_id,ds,y
0,Australia,1998-01-01,23182.197269
1,Australia,1998-04-01,20323.380067
2,Australia,1998-07-01,19826.640511
3,Australia,1998-10-01,20830.129891
4,Australia,1999-01-01,22087.353380
...,...,...,...
33987,Australia/Western Australia/Experience Perth/V...,2014-10-01,381.616236
33988,Australia/Western Australia/Experience Perth/V...,2015-01-01,525.988913
33989,Australia/Western Australia/Experience Perth/V...,2015-04-01,286.874141
33990,Australia/Western Australia/Experience Perth/V...,2015-07-01,439.624378


In [31]:
# realizar consolidacion temporal (de la data todos los niveles geográficos, ahora también se expande a todas las agregaciones temporales)
Y_train_df_te, S_train_df_te, tags_te_train = aggregate_temporal(df = Y_train_df_cs, spec = spec_temporal)
Y_test_df_te, S_test_df_te, tags_te_test = aggregate_temporal(df = Y_test_df_cs, spec = spec_temporal)

In [32]:
Y_test_df_te

,temporal_id,unique_id,ds,y
0,year-1,Australia,2016-10-01,101484.586551
1,year-2,Australia,2017-10-01,107709.864650
2,year-1,Australia/ACT,2016-10-01,2457.401367
3,year-2,Australia/ACT,2017-10-01,2734.748452
4,year-1,Australia/ACT/Business,2016-10-01,754.139245
...,...,...,...,...
5945,quarter-4,Australia/Western Australia/Visiting,2016-10-01,787.030391
5946,quarter-5,Australia/Western Australia/Visiting,2017-01-01,702.777251
5947,quarter-6,Australia/Western Australia/Visiting,2017-04-01,642.516090
5948,quarter-7,Australia/Western Australia/Visiting,2017-07-01,646.521395


In [33]:
# ojo aunque ahora hayan más series geográficas la matriz S para agregacion temporal siendo siendo la misma 
# (existiendo más o menos series geográficas)
S_train_df_te

,temporal_id,quarter-1,quarter-2,quarter-3,quarter-4,quarter-5,quarter-6,quarter-7,quarter-8,quarter-9,...,quarter-63,quarter-64,quarter-65,quarter-66,quarter-67,quarter-68,quarter-69,quarter-70,quarter-71,quarter-72
0,year-1,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,year-2,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,year-3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,year-4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,year-5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,quarter-68,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
122,quarter-69,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
123,quarter-70,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
124,quarter-71,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


If you don’t have a test set available, as is usually the case when you’re making forecasts, it is necessary to create a future dataframe that holds the correct bottom-level unique_ids and timestamps so that they can be temporally aggregated. We can use the make_future_dataframe helper function for that.

In [34]:
from hierarchicalforecast.utils import make_future_dataframe

In [35]:
Y_test_df_te_new = make_future_dataframe(Y_train_df_te, freq="QS", h=horizon)
Y_test_df_te_new, S_test_df_te_new, tags_te_test_new = aggregate_temporal(df=Y_test_df_te_new, spec=spec_temporal)
Y_test_df_te_new.head(3)

,temporal_id,unique_id,ds
0,year-1,Australia,2016-10-01
1,year-2,Australia,2017-10-01
2,year-1,Australia/ACT,2016-10-01


In [36]:
Y_test_df_te_new.shape

(5950, 3)

### 8. Computing base forecasts - FCST AGREGACION TEMPORAL (para todas las combinaciones geográficas, ahora también agregacion temporal - año/semestre/quarter)

**Note also that both frequency and horizon are different for each temporal aggregation. In this example, the lowest level has a quarterly frequency, and a horizon of 8 (constituting 2 years). The year aggregation thus has a yearly frequency with a horizon of 2.**

In [37]:
Y_hat_dfs_te = []
id_cols = ["unique_id", "temporal_id", "ds", "y"]
# We will train a model for each temporal level
for level, temporal_ids_train in tags_te_train.items():
    # Filter the data for the level
    Y_level_train = Y_train_df_te.query("temporal_id in @temporal_ids_train")
    temporal_ids_test = tags_te_test[level]
    Y_level_test = Y_test_df_te.query("temporal_id in @temporal_ids_test")
    # For each temporal level we have a different frequency and forecast horizon
    freq_level = pd.infer_freq(Y_level_train["ds"].unique())
    horizon_level = Y_level_test["ds"].nunique()
    # Train a model and create forecasts
    fcst = StatsForecast(models=[AutoETS(model='ZZZ')], freq=freq_level, n_jobs=-1)
    Y_hat_df_te_level = fcst.forecast(df=Y_level_train[["ds", "unique_id", "y"]], h=horizon_level)
    # Add the test set to the forecast
    Y_hat_df_te_level = Y_hat_df_te_level.merge(Y_level_test, on=["ds", "unique_id"], how="left")
    # Put cols in the right order (for readability)
    Y_hat_cols = id_cols + [col for col in Y_hat_df_te_level.columns if col not in id_cols]
    Y_hat_df_te_level = Y_hat_df_te_level[Y_hat_cols]
    # Append the forecast to the list
    Y_hat_dfs_te.append(Y_hat_df_te_level)

Y_hat_df_te = pd.concat(Y_hat_dfs_te, ignore_index=True)

In [38]:
Y_hat_df_te.head()

,unique_id,temporal_id,ds,y,AutoETS
0,Australia,year-1,2016-10-01,101484.586551,97448.236033
1,Australia,year-2,2017-10-01,107709.864650,97448.236033
2,Australia/ACT,year-1,2016-10-01,2457.401367,2295.142976
3,Australia/ACT,year-2,2017-10-01,2734.748452,2295.142976
4,Australia/ACT/Business,year-1,2016-10-01,754.139245,616.497987


### 9. Reconciliación temporal 
#### (DUDA SI SE HACE conciliación TEMPORAL, nada asegura que se mantenga la conciliación GEOGRÁFICA???)

In [39]:
reconcilers = [
    BottomUp(),
    MinTrace(method='ols')
]
hrec = HierarchicalReconciliation(reconcilers=reconcilers)
Y_rec_df_te = hrec.reconcile(Y_hat_df = Y_hat_df_te, S = S_test_df_te, tags = tags_te_test, temporal = True)

#### 9.a Validar que se conserva reconciliación geográfica (está la duda porque en ninguna parte se usa el fcst reconciliación geográfica)

In [40]:
# obtener las series de "Country"
list_series_country = tags_cs['Country'].tolist()
list_series_country

['Australia']

In [41]:
# obtener las series 'Country/State/Region/Purpose' (EL NIVEL DE MAYOR DESAGREGACIÓN DEL FORECAST)
list_series_all_desagregacion = tags_cs['Country/State/Region/Purpose'].tolist()
len(list_series_all_desagregacion)

304

In [42]:
# definir fecha de fcst para evaluar las diferentes agregaciones geográficas
fecha_example_filter = '2016-01-01'

In [43]:
# filtrar forecast país para una fecha en específico (para comparar vs la suma del forecast cada state por separado y luego sumándolo)
# USAR DF conciliación temporal
mask_example_country = (Y_rec_df_te['unique_id'].isin(list_series_country)) & (Y_rec_df_te['ds'] == fecha_example_filter)
Y_rec_df_te[mask_example_country]

,unique_id,temporal_id,ds,y,AutoETS,AutoETS/BottomUp,AutoETS/MinTrace_method-ols
2550,Australia,quarter-1,2016-01-01,26660.637689,24088.540395,24088.540395,24211.417015


In [44]:
# FILTRAR FORECAST DEL NIVEL DE MAYOR DESEGREGACION Y SUMAR HASTA LLEGAR A COUNTRY. validar que bottom/up sumó desde el nivel de mayor desagregacion
mask_example_all_desagregation = (Y_rec_df_te['unique_id'].isin(list_series_all_desagregacion)) & (Y_rec_df_te['ds'] == fecha_example_filter)
Y_rec_df_te_example = Y_rec_df_te[mask_example_all_desagregation]

cols_to_sum = ['AutoETS', 'AutoETS/BottomUp', 'AutoETS/MinTrace_method-ols']
Y_rec_df_te_example[cols_to_sum].sum(axis=0).to_frame(name='sum_result').T

,AutoETS,AutoETS/BottomUp,AutoETS/MinTrace_method-ols
sum_result,23136.828519,23136.828519,22852.755381


In [45]:
# la suma de las series geográficas para una fecha no da lo mismo que al comparar con el total_país (cuando se hace consolidación temporal)
# así se necesita elegir una

# EFECTIVAMENTE NO COINCIDE. 
# ENTONCES SOLO SE PUEDE HACER UNA SOLA CONSOLIDACIÓN.

### NOTEBOOK BASE CONTINUA CON LA EVALUACIÓN
https://nixtlaverse.nixtla.io/hierarchicalforecast/examples/australiandomestictourismcrosstemporal.html#4-evaluation

In [46]:
# se omite porque se puede ver que en el ejemplo, al hacer consolidación temporal no se mantiene la consolidación geográfica
# se tiene que elegir una o la otra